## 🪄 Install `raga-testing-platform` library

In [ ]:
pip install raga-testing-platform==1.1.49

### Put your `ACCESS KEY & SECRET KEY`

In [ ]:
# Define the access key, secret key, and host
ACCESS_KEY = "Paste_your_access_key_here"
SECRET_KEY = "Paste_your_secret_key_here"
HOST = "https://backend.prod3.ragaai.ai"

### Import All raga lib from raga module

In [ ]:
from raga import *
import datetime

### 🪄 Set up a Run

In [ ]:
project_name = "Object Detection"
run_name = f"Demo-Run-Pipeline-Tests-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
test_session = TestSession(project_name=project_name,run_name=run_name, access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST)

## 🚀 Pipeline Tests

### 👟 Run Event Detection - A/B Testing

In [ ]:
dataset_name = "Enter_dataset_name"

rules = EventABTestRules()
rules.add(metric="difference_percentage", IoU=0.5, _class="ALL", threshold=50.0, conf_threshold=0.2)

filters = Filter()
filters.add(TimestampFilter(gte="2021-01-01T00:00:00Z", lte="2025-01-15T00:00:00Z"))

model_comparison_check = event_ab_test(test_session=test_session,
                                       dataset_name=dataset_name,
                                       test_name="AB-test-unlabelled",
                                       modelB="ModelB",
                                       modelA="ModelA",
                                       object_detection_modelB="ModelB",
                                       object_detection_modelA="ModelA",
                                       type="metadata",
                                       sub_type="unlabelled",
                                       output_type="event_detection",
                                       rules=rules,
                                       aggregation_level=["weather"],
                                       filter=filters)
test_session.add(model_comparison_check)
test_session.run()

### 👟 Run Event Detection - FMA

In [ ]:
dataset_name = "Enter_dataset_name"

rules = FMARules()
rules.add(metric="Precision", conf_threshold=0.2, metric_threshold=0.5, iou_threshold=0.5, frame_overlap_threshold=0.5, label="ALL")
rules.add(metric="Recall", conf_threshold=0.2, metric_threshold=0.5, iou_threshold=0.5, frame_overlap_threshold=0.5, label="ALL")

edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name = dataset_name,
                                            test_name = "FMAEventMD",
                                            gt="GT",
                                            model="Model",
                                            object_detection_gt="GT",
                                            object_detection_model="Model",
                                            rules = rules,
                                            output_type="event_detection",
                                            type="metadata",
                                            aggregation_level=["weather","time_of_day"]
                                            )

test_session.add(edge_case_detection)
test_session.run()

### 🪄 Set up a Run

In [ ]:
project_name = "Object Detection"
run_name = f"Demo-Run-FasterRCNN-Tests-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"
test_session = TestSession(project_name=project_name,run_name=run_name, access_key=ACCESS_KEY, secret_key=SECRET_KEY, host=HOST)

## 🚀 FasterRCNN Tests

### 👟 Object Detection - Drift	

In [ ]:
train_dataset_name = "Enter_train_dataset_name"
field_dataset_name = "Enter_field_dataset_name"

rules = DriftDetectionRules()
rules.add(type="drift_detection", dist_metric="Mahalanobis", _class="ALL", threshold=2)

edge_case_detection = data_drift_detection(test_session=test_session,
                                            test_name=f"Drift-Detection-Test",
                                            train_dataset_name=train_dataset_name,
                                            field_dataset_name=field_dataset_name,
                                            train_embed_col_name="ImageVectorsM1",
                                            field_embed_col_name = "ImageVectorsM1",
                                            level = "image",
                                            rules = rules)

test_session.add(edge_case_detection)
test_session.run()

### 👟 Run Object Detection - Outlier

In [ ]:
dataset_name = "Enter_dataset_name"

rules = DriftDetectionRules()
rules.add(type="anomaly_detection", dist_metric="Mahalanobis", _class="ALL", threshold=25)

edge_case_detection = data_drift_detection(test_session=test_session,
                                            test_name=f"Outlier_Detection",
                                            dataset_name=dataset_name,
                                            embed_col_name = "ImageVectorsM1",
                                            output_type = "outlier_detection",
                                            rules = rules)

test_session.add(edge_case_detection)
test_session.run()

### 👟 Run Data Leakage Test

In [ ]:
train_dataset_name = "Enter_train_dataset_name"
field_dataset_name = "Enter_field_dataset_name"

rules = DLRules()
rules.add(metric = 'overlapping_samples', metric_threshold = 0.92)

edge_case_detection = data_leakage_test(test_session=test_session,
                                            test_name="Data-Leakage-Test",
                                            train_dataset_name=train_dataset_name,
                                            dataset_name=field_dataset_name,
                                            type = "data_leakage",
                                            output_type="image_data",
                                            train_embed_col_name="ImageVectorsM1",
                                            embed_col_name = "ImageVectorsM1",
                                            rules = rules)

test_session.add(edge_case_detection)
test_session.run()

### 👟 Run Object Detection - FMA

In [ ]:
dataset_name = "Enter_dataset_name"

rules = FMARules()
rules.add(metric="Precision", conf_threshold=0.4, metric_threshold=0.50, iou_threshold=0.4, label="ALL")
rules.add(metric="F1Score", conf_threshold=0.4, metric_threshold=0.50, iou_threshold=0.4, label="ALL")
rules.add(metric="Recall", conf_threshold=0.4, metric_threshold=0.50, iou_threshold=0.4, label="ALL")

cls_default = clustering(test_session=test_session, 
                         dataset_name = dataset_name, 
                         method="k-means", 
                         embedding_col="ImageVectorsM1", 
                         level="image", 
                         args= {"numOfClusters": 6}, 
                         force=True)

edge_case_detection = failure_mode_analysis(test_session=test_session,
                                            dataset_name = dataset_name,
                                            test_name = "Failure Mode Analysis",
                                            model = "ModelA",
                                            gt = "GT",
                                            rules = rules,
                                            output_type="object_detection",
                                            type="embedding",
                                            clustering=cls_default
                                            )

test_session.add(edge_case_detection)
test_session.run()

Please switch to https://prod3.ragaai.ai/home/project/runs?projectId=51